In [1]:
import csv
file_path = r"IMDB Dataset.csv"
review = []
sentiment = []
with open(file_path,'r',encoding='utf-8') as file:
    reader = csv.reader(file,delimiter=',')
    for line in reader:
        review.append(line[0])
        sentiment.append(line[1])
        


In [2]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'review':review,'sentiment':sentiment})

In [3]:
df.shape

(50000, 2)

In [4]:
for i in df['review']:
    if '<br />' in i:
        i.replace('<br />','')

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(oov_token="Abhinav")
tokenizer.fit_on_texts(df['review'])


In [6]:
tokenizer.word_index

{'Abhinav': 1,
 'the': 2,
 'and': 3,
 'a': 4,
 'of': 5,
 'to': 6,
 'is': 7,
 'br': 8,
 'in': 9,
 'it': 10,
 'i': 11,
 'this': 12,
 'that': 13,
 'was': 14,
 'as': 15,
 'for': 16,
 'with': 17,
 'movie': 18,
 'but': 19,
 'film': 20,
 'on': 21,
 'not': 22,
 'you': 23,
 'are': 24,
 'his': 25,
 'have': 26,
 'be': 27,
 'one': 28,
 'he': 29,
 'all': 30,
 'at': 31,
 'by': 32,
 'an': 33,
 'they': 34,
 'so': 35,
 'who': 36,
 'from': 37,
 'like': 38,
 'or': 39,
 'just': 40,
 'her': 41,
 'out': 42,
 'about': 43,
 'if': 44,
 "it's": 45,
 'has': 46,
 'there': 47,
 'some': 48,
 'what': 49,
 'good': 50,
 'when': 51,
 'more': 52,
 'very': 53,
 'up': 54,
 'no': 55,
 'time': 56,
 'my': 57,
 'even': 58,
 'would': 59,
 'she': 60,
 'which': 61,
 'only': 62,
 'really': 63,
 'see': 64,
 'story': 65,
 'their': 66,
 'had': 67,
 'can': 68,
 'me': 69,
 'well': 70,
 'were': 71,
 'than': 72,
 'much': 73,
 'we': 74,
 'bad': 75,
 'been': 76,
 'get': 77,
 'do': 78,
 'great': 79,
 'other': 80,
 'will': 81,
 'also': 82,


In [7]:
numWords = len(tokenizer.word_index)

In [8]:
df['review'] = tokenizer.texts_to_sequences(df['review'])

In [9]:
df['review'][0]

[28,
 5,
 2,
 80,
 2103,
 46,
 1073,
 13,
 101,
 148,
 40,
 308,
 3185,
 399,
 475,
 27,
 3196,
 34,
 24,
 204,
 15,
 12,
 7,
 622,
 49,
 597,
 17,
 69,
 8,
 8,
 2,
 87,
 149,
 13,
 3242,
 69,
 43,
 3185,
 14,
 93,
 5399,
 3,
 15116,
 135,
 5,
 571,
 61,
 269,
 9,
 204,
 37,
 2,
 662,
 140,
 1741,
 69,
 12,
 7,
 22,
 4,
 120,
 16,
 2,
 7889,
 2334,
 39,
 11470,
 12,
 120,
 2596,
 55,
 5912,
 17,
 5511,
 6,
 1480,
 377,
 39,
 571,
 93,
 7,
 3805,
 9,
 2,
 361,
 357,
 5,
 2,
 662,
 8,
 8,
 10,
 7,
 434,
 3185,
 15,
 13,
 7,
 2,
 11649,
 359,
 6,
 2,
 15702,
 6814,
 2539,
 1065,
 56310,
 10,
 2712,
 1422,
 21,
 23547,
 539,
 33,
 4637,
 2469,
 5,
 2,
 1209,
 118,
 30,
 2,
 7018,
 26,
 2971,
 11965,
 3,
 392,
 56311,
 35,
 16741,
 7,
 22,
 300,
 21,
 2,
 4911,
 7365,
 539,
 7,
 345,
 6,
 107,
 25422,
 8162,
 42637,
 14814,
 5051,
 7890,
 2454,
 3,
 52,
 35,
 48043,
 328,
 9107,
 7366,
 12487,
 3,
 8698,
 33192,
 24,
 111,
 226,
 244,
 8,
 8,
 11,
 59,
 132,
 2,
 281,
 1325,
 5,
 2,
 120,
 

In [10]:
import pickle
with open('tokenizer.pkl','wb') as file:
    pickle.dump(tokenizer,file)

In [11]:
from tensorflow.keras.utils import pad_sequences
padded = pad_sequences(df['review'],maxlen=236 ,padding='post')

In [12]:
padded.size

11800000

In [13]:
df['sentiment'] = np.where(df['sentiment']=='negative',0,1)

In [14]:
from tensorflow import keras
from keras import Sequential
from keras.layers import LSTM,Embedding,Dense,GRU,Bidirectional

In [15]:
model = Sequential()
model.add(Embedding(input_dim=numWords+1,output_dim=16))
model.add(Bidirectional(LSTM(32,return_sequences=True)))
model.add(Bidirectional(LSTM(8)))
model.add(Dense(8,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.build(input_shape=(None,236))

In [16]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 236, 16)        │     1,988,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 236, 64)        │        12,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 16)             │         4,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,005,425 (7.65 MB)

 Trainable params: 2,005,425 (7.65 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.fit(padded,df['sentiment'],epochs=5,validation_split=0.2)

Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 72s 56ms/step - accuracy: 0.6968 - loss: 0.5547 - val_accuracy: 0.8457 - val_loss: 0.3688
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 79s 63ms/step - accuracy: 0.9169 - loss: 0.2334 - val_accuracy: 0.8742 - val_loss: 0.3057
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 84s 67ms/step - accuracy: 0.9484 - loss: 0.1539 - val_accuracy: 0.8773 - val_loss: 0.3479
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 77s 62ms/step - accuracy: 0.9681 - loss: 0.1018 - val_accuracy: 0.8484 - val_loss: 0.4078
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 75s 60ms/step - accuracy: 0.9706 - loss: 0.0881 - val_accuracy: 0.8623 - val_loss: 0.4236


In [19]:
from keras.models import save_model
save_model(model,'model.keras')

In [20]:
model.output_shape

(None, 1)

In [21]:
model.input_shape

(None, 236)